# LiteLLM Integration with Maxim SDK Tracing

This tutorial demonstrates how to integrate Maxim SDK tracing capabilities with LiteLLM. You'll learn how to set up and configure Maxim's tracing functionality to monitor and analyze your LiteLLM API calls, providing valuable insights into your LLM application's performance and usage patterns.


## Step 1: Initializing Maxim SDK and adding as a logger in litellm

In [ ]:
import litellm
from maxim import Maxim
from maxim.logger.litellm import MaximLiteLLMTracer

logger = Maxim().logger()
# This is the single line integration of Maxim SDK with LiteLLM
litellm.callbacks = [MaximLiteLLMTracer(logger)]

## Step 2: Initializing litellm router

In [2]:
import os
from litellm import acompletion, completion

In [ ]:
# Sync completion
response = completion(
        model="openai/gpt-4o",
        api_key=os.getenv("OPENAI_API_KEY"),
        messages=[{"role": "user", "content": "Hello, world!"}],        
    )

print(response.choices[0].message.content)

In [ ]:
# Async completion
response = await acompletion(
        model="openai/gpt-4o",
        api_key=os.getenv("OPENAI_API_KEY"),
        messages=[{"role": "user", "content": "Hello, world!"}],        
    )

print(response.choices[0].message.content)

In [ ]:
# Vision model
response = await acompletion(
        model="openai/gpt-4o",
        api_key=os.getenv("OPENAI_API_KEY"),
        messages=[{"role": "user", "content": [
            {
                "type":"text",
                "text": "Explain this image in 3 points"
            },
            {
                "type":"image_url",
                "image_url": {
                    "url": "https://upload.wikimedia.org/wikipedia/commons/4/47/PNG_transparency_demonstration_1.png"
                }
            }
        ]}],        
    )

print(response.choices[0].message.content)

## Attaching a custom trace to the litellm generation

In [ ]:
from maxim.logger.logger import TraceConfig
import uuid

trace = logger.trace(TraceConfig(id=str(uuid.uuid4()), name="litellm-generation"))
trace.event(str(uuid.uuid4()), "litellm-generation", "litellm-generation", {})
# This will attach litellm generation to the trace inside a span.
response = await acompletion(
        model="openai/gpt-4o",
        api_key=os.getenv("OPENAI_API_KEY"),
        messages=[{"role": "user", "content": "What can you do for me!"}],        
        metadata={"maxim": {"trace_id": trace.id, "span_name": "litellm-generation"}}
    )

print(response.choices[0].message.content)

# ![LiteLLM Custom Trace](images/litellm-custom-trace.png)


